# Summary & Participant Exclusion
Status: Implemented
 -- Fully automatized

In [1]:
from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
# add gaze?

In [3]:
snippets_data = pd.read_csv(SNIPPETS_PATH, sep=SEPARATOR).fillna(0)
snippets_data[SNIPPET_COLUMN_SNIPPET] = snippets_data[SNIPPET_COLUMN_SNIPPET].str.replace(
    '\'', '')
snippets_data[SNIPPET_COLUMN_SNIPPET] = snippets_data[SNIPPET_COLUMN_SNIPPET].apply(
    lambda value: value.split('.')[0])
print(snippets_data[SNIPPET_COLUMN_SNIPPET])

0      4-clean-v0
1      4-clean-v1
2      4-clean-v2
3        4-obf-v0
4        4-obf-v1
          ...    
139     -clean-v1
140     -clean-v2
141       -obf-v0
142       -obf-v1
143       -obf-v2
Name: filename, Length: 144, dtype: object


### Summarize all exclusion results
* summarize, also per category / per participant
* check whether a participant has less than x(10 probably) trials not excluded --> if too little, exclude completely

In [4]:
# summary BEHAVIORAL
#  summary EEG & VISUAL together based on BEHAVIORAL, but also separate
# per participant and snippet
behavioral_exclusion_summary = pd.DataFrame([], columns=[
                                            participant for participant in folders], index=snippets_data[SNIPPET_COLUMN_SNIPPET])
all_exclusion_summary = pd.DataFrame([], columns=[
                                     participant for participant in folders], index=snippets_data[SNIPPET_COLUMN_SNIPPET])
all_exclusion_summary_plus_violations = pd.DataFrame([], columns=[
                                     participant for participant in folders], index=snippets_data[SNIPPET_COLUMN_SNIPPET])
for participant in tqdm(folders):
    print('----------------------------------------------')
    print(PARTICIPANT, participant)
    # participant marker
    e_p = get_exclusions(participant, [PARTICIPANT])[PARTICIPANT]
    if e_p[BEHAVIORAL]:
        behavioral_exclusion_summary[participant] = False
    if e_p[BEHAVIORAL] or e_p[EEG] or e_p[VISUAL]:
        all_exclusion_summary[participant] = False
        all_exclusion_summary_plus_violations[participant] = False

    # snippet marker
    e_s = get_exclusions(participant, [SNIPPET])[SNIPPET]
    snippet_violations_path = get_eeg_violations_path(participant)
    if snippet_violations_path.exists():
        snippet_violations = pd.read_csv(
            snippet_violations_path, sep=SEPARATOR, index_col=0, dtype={PARTICIPANT: str})
        snippet_violations = snippet_violations.dropna(axis=0, how='all')
    for snippet in e_s:
        behavioral_exclusion_summary.loc[snippet, participant] = e_s[snippet][BEHAVIORAL] if e_s[snippet][BEHAVIORAL] else True
        exclusion_reasons = e_s[snippet][BEHAVIORAL] + \
            e_s[snippet][EEG] + e_s[snippet][VISUAL]
        if not exclusion_reasons:
            value = True
        else:
            value = str(exclusion_reasons)
        all_exclusion_summary.loc[snippet, participant] = value
        if snippet_violations_path.exists() and (snippet in snippet_violations.index):
            eeg_violations = snippet_violations.loc[snippet].dropna().to_dict()
            exclusion_reasons += [eeg_violations]
        if not exclusion_reasons:
            value = True
        else:
            value = str(exclusion_reasons)
        all_exclusion_summary_plus_violations.loc[snippet, participant] = value

behavioral_exclusion_summary.to_csv(
    get_statistic_summary_path('behavioral'), sep=SEPARATOR)
all_exclusion_summary.to_csv(get_statistic_summary_path('all'), sep=SEPARATOR)
all_exclusion_summary_plus_violations.to_csv(get_statistic_summary_path('including EEG violations'), sep=SEPARATOR)

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
----------------------------------------------
Participant 004
----------------------------------------------
Participant 005
----------------------------------------------
Participant 007
----------------------------------------------
Participant 008
----------------------------------------------
Participant 009
----------------------------------------------
Participant 010
----------------------------------------------
Participant 011
----------------------------------------------
Participant 012
----------------------------------------------
Participant 013
----------------------------------------------
Participant 014
----------------------------------------------
Participant 015
----------------------------------------------
Participant 016
----------------------------------------------
Participant 017
----------------------------------------------
Participant 018
----------------------------------------------
Particip